# Chargement d'un fichier

Toutefois prendre note que l'opération textFile ne fait aucun accès ou chargement.
En réalité, ca crée un RDD et on peut y appliquer une multitude transformations sans traitement réel.
C'est l'utilisation d'une action qui déclenche le traitement.
Voici des exemples d'actions: count, top, 

Pointer un fichier dans un RDD et compter le nombre de lignes.


In [4]:
acces= sc.textFile("/mnt/ul/ul-mpo-ts-wls01/depot_journal/mpo-testpr/mpo-server-testpr01/mpo-portail/acces.log")
acces.count()

9016


Regarder une ligne à titre d'exemple.


In [5]:
acces.top(1)

[u'2016-06-10 15:35:43\t200\tanonyme\tGET\thttps://monportail.testpr.ulaval.ca:443/auth/deconnecter\t6ms\t10.12.33.146\t420ea147-97bb-454e-a0bb-270af851e1c3\tMozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36']


Séparer les éléments via le séparateur de tabulation.


In [6]:
ip = acces.map(lambda x: x.split("\t")[6])
ip.count()

9016

In [7]:
ip.top(10)

[u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177',
 u'132.203.52.177']

Grouper les adresses IP par classe A.

In [8]:
classA = ip.groupBy(lambda x: x.split(".")[0])

In [10]:
classA.count()

2

Que sont ces classes A ?

In [11]:
classA.top(10)

[(u'132', <pyspark.resultiterable.ResultIterable at 0x7f394134fe10>),
 (u'10', <pyspark.resultiterable.ResultIterable at 0x7f394135d1d0>)]

Hum, ok, les entrées du groupement sont dans un itérateur imbriqué.
Regardons voir le contenu de ces itérateurs.

In [12]:
result = classA.collect()

for (key, values) in result:
    print ("*" * 115) + "\n"
    print ("* Classe A: " + key)
    print (values.data)
    

*******************************************************************************************************************

* Classe A: 10
[u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'10.12.33.84', u'1


Oh! Les duplicats ne sont pas enlevés des valeurs groupées.
Pouvons nous les rendre unique?


Oui: avant de procéder au groupement, on rend les IP uniques.

In [13]:
classA = ip.distinct().groupBy(lambda x: x.split(".")[0])
classA.count()

2

In [14]:
result = classA.collect()

for (key, values) in result:
    print ("*" * 115) + "\n"
    print ("* Classe A: " + key)
    print (values.data)

*******************************************************************************************************************

* Classe A: 10
[u'10.12.33.178', u'10.12.33.136', u'10.12.33.138', u'10.12.33.156', u'10.248.192.254', u'10.248.57.149', u'10.12.33.183', u'10.12.33.202', u'10.248.61.252', u'10.40.29.49', u'10.12.33.123', u'10.12.33.121', u'10.12.33.169', u'10.12.33.149', u'10.248.62.89', u'10.248.82.94', u'10.248.156.164', u'10.12.33.196', u'10.12.33.194', u'10.12.33.190', u'10.12.33.49', u'10.12.33.215', u'10.248.120.2', u'10.248.124.166', u'10.12.33.153', u'10.248.86.186', u'10.248.15.205', u'10.248.58.94', u'10.12.33.55', u'10.248.25.150', u'10.12.33.97', u'10.12.33.223', u'10.248.96.112', u'10.248.174.219', u'10.248.10.176', u'10.248.206.226', u'10.12.33.146', u'10.12.33.140', u'10.12.33.166', u'10.12.33.148', u'10.248.181.14', u'10.40.28.45', u'10.12.33.117', u'10.40.103.13', u'10.248.80.1', u'10.12.33.48', u'10.12.33.68', u'10.12.33.86', u'10.12.33.84', u'10.248.58.161', u'10.248

Mieux non?


#  Pairs RDD
Un RDD ne se compose pas seulement d'une colonne.. 
On peut utiliser les pairsRdds.

In [15]:
pairRDD = ip.map(lambda x: (x,1))
pairRDD.top(10)
#grouped = pairRDD.groupByKey().top(10)
#for (key, value) in grouped:
#    print "***" + key + "***"
#    for val in value:
#        print val


    

[(u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1),
 (u'132.203.52.177', 1)]

Tenter de décompter le nombre de requête par adresse IP, on se sert du pair RDD qui a 1 pour chaque entrée.

In [143]:
pairRDD.reduceByKey(lambda x,y: x+y).top(10)

[(u'132.203.52.177', 121),
 (u'132.203.52.126', 2),
 (u'132.203.243.5', 5),
 (u'132.203.243.4', 5),
 (u'132.203.240.99', 3),
 (u'10.40.29.49', 4),
 (u'10.40.103.13', 5),
 (u'10.248.86.186', 221),
 (u'10.248.82.94', 18),
 (u'10.248.80.1', 22)]

OK!

# Trouver le nombre d'accès par IP

